In [ ]:
# --- Definitive Installation for PyTorch and Mamba in Colab/Kaggle ---
# This process ensures full compatibility between the libraries.

# 1. Completely uninstall existing versions to prevent conflicts.
# The '-y' flag automatically confirms the uninstall.
print("--- Step 1: Uninstalling existing torch and mamba libraries ---")
!pip uninstall -y torch torchvision torchaudio mamba-ssm causal-conv1d

# 2. Install a specific, known-good version of PyTorch compatible with Colab's CUDA drivers.
print("\n--- Step 2: Installing a compatible version of PyTorch ---")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 3. Re-install Mamba from source. This will now compile against the new PyTorch version.
print("\n--- Step 3: Compiling and installing Mamba from source ---")
!git clone https://github.com/state-spaces/mamba /content/mamba
%cd /content/mamba
!pip install .
%cd /content/
print("DONE!")

In [ ]:
!pip install mamba-ssm[causal-conv1d] --no-build-isolation
!pip install triton

In [ ]:
!pip3 install -q -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [4]:
# --- For the FastAPI Server ---
import fastapi
import uvicorn
from pydantic import BaseModel

# --- For PyTorch and the AI Model ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
from transformers import AutoTokenizer
from faker import Faker
# --- For Simulation ---
import logging
import random
import string
import math

# --- Check Device ---
device = torch.device("cpu")
print(f"Using device: {device}")

# --- 1. Define the Fed-Mamba-SCL Architecture (Must match training) ---
# --- 3. The Fed-Mamba-SCL Architecture ---

class Mamba(nn.Module):
    def __init__(self, d_model, d_state=16, d_conv=4, expand=2):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_conv = d_conv
        self.expand = expand
        self.d_inner = int(self.expand * self.d_model)
        self.dt_rank = math.ceil(self.d_model / 16)

        self.in_proj = nn.Linear(self.d_model, self.d_inner * 2, bias=False)
        self.conv1d = nn.Conv1d(
            in_channels=self.d_inner, out_channels=self.d_inner,
            bias=True, kernel_size=d_conv, groups=self.d_inner, padding=d_conv - 1,
        )
        self.x_proj = nn.Linear(self.d_inner, self.dt_rank + self.d_state * 2, bias=False)
        self.dt_proj = nn.Linear(self.dt_rank, self.d_inner, bias=True)

        A = torch.arange(1, self.d_state + 1, dtype=torch.float32).repeat(self.d_inner, 1)
        self.A_log = nn.Parameter(torch.log(A))
        self.D = nn.Parameter(torch.ones(self.d_inner))
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=False)

    def forward(self, x):
        batch, seq_len, d = x.shape
        x_and_res = self.in_proj(x)
        (x_proj, res) = x_and_res.split(split_size=[self.d_inner, self.d_inner], dim=-1)

        x_conv = x_proj.transpose(1, 2)
        x_conv = self.conv1d(x_conv)[:, :, :seq_len]
        x_conv = x_conv.transpose(1, 2)
        x_conv = F.silu(x_conv)

        x_dbl = self.x_proj(x_conv)
        (delta, B, C) = x_dbl.split(split_size=[self.dt_rank, self.d_state, self.d_state], dim=-1)
        delta = F.softplus(self.dt_proj(delta))
        
        A = -torch.exp(self.A_log.float())
        y = []
        h = torch.zeros(batch, self.d_inner, self.d_state, device=x.device)
        
        for t in range(seq_len):
            dt = delta[:, t, :].unsqueeze(-1)
            b = B[:, t, :].unsqueeze(1)
            c = C[:, t, :].unsqueeze(1)
            u = x_conv[:, t, :].unsqueeze(-1)
            
            dA = torch.exp(dt * A)
            dB = dt * b
            h = dA * h + dB * u
            y_t = (h * c).sum(dim=-1) + self.D * u.squeeze(-1)
            y.append(y_t)
            
        y = torch.stack(y, dim=1)
        y = y * F.silu(res)
        output = self.out_proj(y)
        return output


class FedMambaSCL(nn.Module):
    def __init__(self, vocab_size, num_context_features):
        super().__init__()
        # Must match your trained model config
        self.EMBEDDING_DIM = 640 
        
        self.embedding = nn.Embedding(vocab_size, self.EMBEDDING_DIM)
        
        # Use the CPU-compatible Mamba class defined above
        self.mamba = Mamba(
            d_model=self.EMBEDDING_DIM, 
            d_state=64,
            d_conv=4, 
            expand=4
        )
        
        self.projector = nn.Sequential(
            nn.Linear(self.EMBEDDING_DIM + num_context_features, 256),
            nn.ReLU(),
            nn.Linear(256, 128) 
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(self.EMBEDDING_DIM + num_context_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, context_features):
        embedded_seq = self.embedding(input_ids)
        mamba_output = self.mamba(embedded_seq)
        sequence_summary = mamba_output[:, -1, :] 
        combined_features = torch.cat((sequence_summary, context_features), dim=1)
        pred = self.classifier(combined_features)
        return pred.squeeze(-1)

# Configuration matching the "Large" model
EMBEDDING_DIM = 640
NUM_CONTEXT_FEATURES = 1


# --- 2. Load Resources ---
MODEL_PATH = "/kaggle/input/mamba-fed/pytorch/default/1/fed_mamba_scl" # Update this path to where you uploaded it
SCALER_PATH = "/kaggle/input/mamba-fed/pytorch/default/1/fed_mamba_scl/global_scaler.pkl"          # Update this path

try:
    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    print("Tokenizer loaded.")

    # Load Scaler
    with open(SCALER_PATH, 'rb') as f:
        scaler = pickle.load(f)
    print("Scaler loaded.")

    # Load Model
    model = FedMambaSCL(vocab_size=len(tokenizer), num_context_features=NUM_CONTEXT_FEATURES)
    model.load_state_dict(torch.load(f"{MODEL_PATH}/model_state_dict.pth", map_location=device))
    model.to(device)
    model.eval()
    print("Fed-Mamba-SCL Model loaded successfully.")
    MODEL_READY = True
except Exception as e:
    print(f"CRITICAL ERROR LOADING MODEL: {e}")
    MODEL_READY = False

# --- 3. The Prediction Logic ---
def is_request_malicious(query_text: str, flow_duration_ms: int) -> bool:
    if not MODEL_READY: return False
    
    try:
        # 1. Preprocess Text
        inputs = tokenizer(query_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        input_ids = inputs['input_ids'].to(device)
        
        # 2. Preprocess Context (Flow Duration)
        # Note: Must reshape to (1, 1) for scaler
        context_val = np.array([[flow_duration_ms]])
        context_scaled = scaler.transform(context_val)
        context_tensor = torch.tensor(context_scaled, dtype=torch.float32).to(device)
        
        # 3. Inference
        with torch.no_grad():
            output_prob = model(input_ids, context_tensor).item()
            
        is_attack = output_prob > 0.5
        logging.info(f"Analysis: Text='{query_text[:20]}...' | Dur={flow_duration_ms}ms | Score={output_prob:.4f} | Verdict={'MALICIOUS' if is_attack else 'SAFE'}")
        return is_attack
        
    except Exception as e:
        logging.error(f"Inference Error: {e}")
        return False

Using device: cpu
Tokenizer loaded.
Scaler loaded.
Fed-Mamba-SCL Model loaded successfully.


In [5]:
# --- Configuration for our database ---
DB_FILE = "employees.db"
TABLE_NAME = "employees"
NUM_EMPLOYEES = 1000
import os
import sqlite3
from tqdm.notebook import tqdm  # Best for Kaggle/Jupyter

def create_synthetic_db():
    logging.basicConfig(level=logging.INFO, force=True, format='%(asctime)s - %(levelname)s - %(message)s')
    fake = Faker('en_IN')
    if os.path.exists(DB_FILE):
        os.remove(DB_FILE)
    
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute(f'''
    CREATE TABLE {TABLE_NAME} (
        id INTEGER PRIMARY KEY, name TEXT NOT NULL, email TEXT NOT NULL,
        job_title TEXT, company TEXT, salary INTEGER
    )
    ''')
    
    employees = []
    for i in tqdm(range(1, NUM_EMPLOYEES + 1), desc="Generating synthetic data"):
        employees.append((
            i, fake.name(), fake.email(), fake.job(), fake.company(),
            fake.random_int(min=400000, max=2500000)
        ))
        
    cursor.executemany(f"INSERT INTO {TABLE_NAME} VALUES (?,?,?,?,?,?)", employees)
    conn.commit()
    conn.close()
    logging.info(f"Successfully created database '{DB_FILE}' with {NUM_EMPLOYEES} employee records.")

create_synthetic_db()

Generating synthetic data:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-12-05 14:51:04,848 - INFO - Successfully created database 'employees.db' with 1000 employee records.


In [6]:
from typing import List

app = fastapi.FastAPI(title="ByteGraph-Guard Protected API")
conn = sqlite3.connect("employees.db", check_same_thread=False)
conn.row_factory = sqlite3.Row

class Employee(BaseModel):
    id: int; name: str; email: str; job_title: str; company: str; salary: int
    class Config: from_attributes = True

class SearchResponse(BaseModel):
    count: int; data: List[Employee]

@app.get("/search", response_model=SearchResponse)
async def search_employee(
    name: str, 
    # We read the simulated duration from a custom header
    x_flow_duration: int = fastapi.Header(default=5000, alias="X-Flow-Duration") 
):
    # --- The AI Guard Layer ---
    if is_request_malicious(name, x_flow_duration):
        logging.warning(f"🚫 BLOCKED: Malicious payload detected: '{name}'")
        raise fastapi.HTTPException(status_code=403, detail="ByteGraph-Guard: Request blocked.")

    # --- Business Logic (Only runs if AI says Safe) ---
    logging.info(f"✅ ALLOWED: searching for '{name}'")
    query = "SELECT * FROM employees WHERE name = ?"
    cursor = conn.cursor()
    cursor.execute(query, (name,))
    results = cursor.fetchall()
    employees_found = [Employee.model_validate(row) for row in results]
    return SearchResponse(count=len(employees_found), data=employees_found)

# Server runner code (threading) stays the same...
config = uvicorn.Config(app, host="0.0.0.0", port=5000, log_level="error")
server = uvicorn.Server(config)
import threading
import asyncio
def run_server():
    asyncio.run(server.serve())
    
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
print("Server started on port 5000")

/tmp/ipykernel_1012/3813743183.py:7: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Employee(BaseModel):


Server started on port 5000


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import pickle
import numpy as np
import logging
from transformers import AutoTokenizer

# --- 1. FORCE CPU SETTING ---
# device = torch.device("cpu")
# print(f"⚠️ FORCING SIMULATION TO DEVICE: {device}")

# --- 2. The Pure PyTorch Mamba Class (CPU Compatible) ---
# DO NOT import mamba_ssm. Use this class instead.

# --- 4. Load Resources ---
MODEL_PATH = "/kaggle/input/mamba-fed/pytorch/default/1/fed_mamba_scl"
SCALER_PATH = "/kaggle/input/mamba-fed/pytorch/default/1/fed_mamba_scl/global_scaler.pkl"

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    with open(SCALER_PATH, 'rb') as f:
        scaler = pickle.load(f)

    model = FedMambaSCL(vocab_size=len(tokenizer), num_context_features=1)
    
    # CRITICAL: Map location ensures GPU weights are loaded onto CPU
    model.load_state_dict(torch.load(f"{MODEL_PATH}/model_state_dict.pth", map_location=device))
    
    model.to(device)
    model.eval()
    logging.info("SUCCESS: 35M Parameter Fed-Mamba-SCL Model loaded on CPU.")
    MODEL_READY = True
except Exception as e:
    logging.error(f"CRITICAL ERROR LOADING MODEL: {e}")
    MODEL_READY = False

# --- 5. Prediction Logic (Fail Closed) ---
def is_request_malicious(query_text: str, flow_duration_ms: int) -> bool:
    if not MODEL_READY: 
        logging.error("Model not ready. Failing CLOSED (Blocking).")
        return True # Block if model is broken
    
    try:
        inputs = tokenizer(query_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        input_ids = inputs['input_ids'].to(device)
        
        context_val = np.array([[flow_duration_ms]])
        context_scaled = scaler.transform(context_val)
        context_tensor = torch.tensor(context_scaled, dtype=torch.float32).to(device)
        
        with torch.no_grad():
            output_prob = model(input_ids, context_tensor).item()
            
        is_attack = output_prob > 0.5
        
        # Logging for visibility
        verdict = "MALICIOUS" if is_attack else "SAFE"
        logging.info(f"Scan: '{query_text[:15]}...' | Dur: {flow_duration_ms}ms | Score: {output_prob:.4f} | Verdict: {verdict}")
        
        return is_attack
        
    except Exception as e:
        logging.error(f"Inference Error: {e}")
        return True # Fail Closed: Block request on error

2025-12-05 14:51:42,348 - INFO - SUCCESS: 35M Parameter Fed-Mamba-SCL Model loaded on CPU.


In [12]:
import requests
import random
import string
import time
import logging

# --- 1. Simulation Configuration ---
SQLI_PAYLOADS = {"Generic": ["' OR 1=1 --", "' UNION SELECT user, pass FROM users--", "1; WAITFOR DELAY '0:0:5'--"]}

STEALTHY_PAYLOADS = [
    "'admin'--", "105-1", "' or 1=1 limit 1 --", "smith' OR '1'='1",
    "1 AND 1=1", "1' and '1'='1", "1024-2", "' OR '1'='1",
    "1*1", "admin' #", "' OR 'x'='x", "1=1", "' OR 1=1--",
    "OR '1'='1'", "' or 'x'='x", "x' or 1=1--", "' OR 1=1 /*",
    "1' OR '1'='1' --", "user1' or 1=1 --", "OR+%271%27%3D%271",
    "' AND user IS NULL; --",
]

SHARED_TEMPLATES = [
    "GET /search?q={query}",
    "POST /login {{'user': '{user}', 'pass': 'password123'}}",
    "GET /api/v1/users?id={user_id}",
    "GET /products/details?productID={product_id}",
]

UNIQUE_BENIGN_TEMPLATES = [
    "GET /products/{category}/{product_id}",
    "POST /api/v1/cart/add {{'item_id': '{item_id}', 'quantity': {quantity}}}",
    "GET /assets/css/main.css", "GET /images/products/{image_name}.jpg",
    "POST /profile/update {{'email': '{email}'}}", "GET /blog/posts?page={page_num}",
]

UNIQUE_MALICIOUS_TEMPLATES = [
    "GET /items/show.php?id={payload}", "GET /user/profile?user_id={payload}",
    "GET /search.aspx?q={payload}", "POST /search {{'keyword': '{payload}'}}",
    "POST /login.php {{'username': '{payload}', 'password': 'password'}}",
    "GET /admin/login?user={payload}", "GET /index.php?page={payload}",
]

def get_random_string(length=10):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

# --- 2. Request Generators ---
def generate_benign_request():
    if random.random() < 0.5:
        template = random.choice(SHARED_TEMPLATES)
    else:
        template = random.choice(UNIQUE_BENIGN_TEMPLATES)
        
    placeholders = {
        'query': get_random_string(15), 'user': get_random_string(8),
        'user_id': str(random.randint(100, 9999)), 'product_id': get_random_string(8),
        'category': random.choice(["electronics", "books", "home"]),
        'item_id': str(random.randint(1000, 9999)), 'quantity': random.randint(1, 5),
        'image_name': get_random_string(12), 'email': "test.user@example.com",
        'page_num': random.randint(1, 20)
    }
    return template.format(**placeholders)

def generate_malicious_request():
    if random.random() < 0.5:
        template = random.choice(SHARED_TEMPLATES)
    else:
        template = random.choice(UNIQUE_MALICIOUS_TEMPLATES)
    
    if random.random() < 0.7:
        payload = random.choice(STEALTHY_PAYLOADS)
    else:
        payload = random.choice(SQLI_PAYLOADS["Generic"])
    
    placeholders = {
        'query': payload, 'user': payload, 'user_id': payload, 
        'product_id': payload, 'payload': payload, 'keyword': payload,
        'username': payload, 'page': payload
    }
    return template.format(**placeholders)

# --- 3. The Simulation Loop ---
def run_simulation(num_requests=30):
    print(f"\n{'='*60}")
    print(f"🚀 LAUNCHING FED-MAMBA-SCL SECURITY SIMULATION ({num_requests} REQ)")
    print(f"{'='*60}")
    
    server_url = "http://127.0.0.1:5000/search"
    
    # Metrics
    total_correct = 0
    total_requests = 0
    stats = {
        "true_pos": 0,  # Attacks caught
        "false_neg": 0, # Attacks missed (Danger)
        "true_neg": 0,  # Valid users allowed
        "false_pos": 0  # Valid users blocked (Annoyance)
    }
    
    for i in range(num_requests):
        # A. Flip a coin: Benign (50%) or Malicious (50%)
        is_attack = random.random() < 0.5
        
        # B. Generate Data & Duration
        if is_attack:
            text = generate_malicious_request()
            # Attacker duration profile (50-8000ms)
            duration = random.randint(50, 8000)
            label = "ATTACK"
        else:
            text = generate_benign_request()
            # Benign duration profile (4000-90000ms)
            duration = random.randint(4000, 90000)
            label = "NORMAL"
            
        # C. Send to Server
        try:
            response = requests.get(
                server_url, 
                params={'name': text}, 
                headers={'X-Flow-Duration': str(duration)}
            )
            
            # D. Evaluate Response
            status = response.status_code
            is_blocked = (status == 403)
            
            # Determine correctness
            if is_attack:
                if is_blocked:
                    verdict = "✅ BLOCKED (Success)"
                    stats["true_pos"] += 1
                    total_correct += 1
                else:
                    verdict = "❌ MISSED (Danger!)"
                    stats["false_neg"] += 1
            else:
                if not is_blocked:
                    verdict = "✅ ALLOWED (Success)"
                    stats["true_neg"] += 1
                    total_correct += 1
                else:
                    verdict = "⚠️ FALSE POSITIVE (Annoyance)"
                    stats["false_pos"] += 1

            total_requests += 1
            current_accuracy = (total_correct / total_requests) * 100
            
            # Print Log
            print(f"[{i+1:02}] {label:<7} | Dur: {duration:<5} | {verdict} | Acc: {current_accuracy:.1f}%")
            
        except Exception as e:
            print(f"[{i+1:02}] CRITICAL CONNECTION ERROR: {e}")
            
        time.sleep(0.1) 

    # --- Final Score ---
    accuracy = (total_correct / total_requests) * 100
    print(f"\n{'='*60}")
    print("SIMULATION RESULTS")
    print(f"Final Accuracy: {accuracy:.2f}%")
    print(f"{'-'*30}")
    print(f"Attacks Caught (Recall):     {stats['true_pos']} / {(stats['true_pos']+stats['false_neg'])}")
    print(f"Benign Allowed (Specificity): {stats['true_neg']} / {(stats['true_neg']+stats['false_pos'])}")
    print(f"False Positives:             {stats['false_pos']}")
    print(f"{'='*60}")

if __name__ == "__main__":
    run_simulation(20)

2025-12-05 15:05:30,215 - INFO - Scan: 'POST /search {'...' | Dur: 3052ms | Score: 1.0000 | Verdict: MALICIOUS
2025-12-05 15:05:30,216 - WARNING - 🚫 BLOCKED: Malicious payload detected: 'POST /search {'keyword': '' UNION SELECT user, pass FROM users--'}'



🚀 LAUNCHING FED-MAMBA-SCL SECURITY SIMULATION (20 REQ)
[01] ATTACK  | Dur: 3052  | ✅ BLOCKED (Success) | Acc: 100.0%


2025-12-05 15:05:30,382 - INFO - Scan: 'GET /api/v1/use...' | Dur: 75488ms | Score: 0.0000 | Verdict: SAFE
2025-12-05 15:05:30,382 - INFO - ✅ ALLOWED: searching for 'GET /api/v1/users?id=2642'
2025-12-05 15:05:30,549 - INFO - Scan: 'GET /index.php?...' | Dur: 5864ms | Score: 1.0000 | Verdict: MALICIOUS
2025-12-05 15:05:30,550 - WARNING - 🚫 BLOCKED: Malicious payload detected: 'GET /index.php?page=' OR 1=1 /*'


[02] NORMAL  | Dur: 75488 | ✅ ALLOWED (Success) | Acc: 100.0%
[03] ATTACK  | Dur: 5864  | ✅ BLOCKED (Success) | Acc: 100.0%


2025-12-05 15:05:30,718 - INFO - Scan: 'GET /images/pro...' | Dur: 74818ms | Score: 0.0000 | Verdict: SAFE
2025-12-05 15:05:30,719 - INFO - ✅ ALLOWED: searching for 'GET /images/products/HTQwjbSELWYD.jpg'
2025-12-05 15:05:30,882 - INFO - Scan: 'GET /products/d...' | Dur: 1443ms | Score: 1.0000 | Verdict: MALICIOUS
2025-12-05 15:05:30,883 - WARNING - 🚫 BLOCKED: Malicious payload detected: 'GET /products/details?productID=admin' #'


...

[16] ATTACK  | Dur: 2306  | ✅ BLOCKED (Success) | Acc: 93.8%
[17] NORMAL  | Dur: 51159 | ✅ ALLOWED (Success) | Acc: 94.1%


2025-12-05 15:05:33,058 - INFO - Scan: 'POST /login {'u...' | Dur: 5940ms | Score: 1.0000 | Verdict: MALICIOUS
2025-12-05 15:05:33,059 - WARNING - 🚫 BLOCKED: Malicious payload detected: 'POST /login {'user': 'Bk95Bjt2', 'pass': 'password123'}'
2025-12-05 15:05:33,223 - INFO - Scan: 'GET /items/show...' | Dur: 6691ms | Score: 1.0000 | Verdict: MALICIOUS
2025-12-05 15:05:33,224 - WARNING - 🚫 BLOCKED: Malicious payload detected: 'GET /items/show.php?id=' OR 'x'='x'


[18] NORMAL  | Dur: 5940  | ⚠️ FALSE POSITIVE (Annoyance) | Acc: 88.9%
[19] ATTACK  | Dur: 6691  | ✅ BLOCKED (Success) | Acc: 89.5%


2025-12-05 15:05:33,395 - INFO - Scan: 'GET /search?q=1...' | Dur: 59924ms | Score: 1.0000 | Verdict: MALICIOUS
2025-12-05 15:05:33,395 - WARNING - 🚫 BLOCKED: Malicious payload detected: 'GET /search?q=1MIZpnkOJULO3rs'


[20] NORMAL  | Dur: 59924 | ⚠️ FALSE POSITIVE (Annoyance) | Acc: 85.0%

SIMULATION RESULTS
Final Accuracy: 85.00%
------------------------------
Attacks Caught (Recall):     11 / 11
Benign Allowed (Specificity): 6 / 9
False Positives:             3
